In [ ]:
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.messages import TextMessage
from autogen_core import CancellationToken
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from dotenv import load_dotenv
import os
import asyncio
load_dotenv()

apikey = os.getenv("OPENAI_API_KEY")

modelclient = OpenAIChatCompletionClient(api_key=apikey, model="gpt-4o-mini")

In [ ]:
assistant = AssistantAgent(name="Assistant",
                           model_client=modelclient,
                           system_message="You are a helpful assistant,  Answer the question."
                           )

result = await assistant.on_messages(
    messages =[TextMessage(role="user",content="Tell me a joke.",source="user")],
    cancellation_token=CancellationToken()
)

print(result.chat_message.content)

#### **Saving the State**

In [ ]:
state_agent = await assistant.save_state()

In [ ]:
print("State saved.: ", state_agent)

#### **Load the State or history to the  other assistant or Another Agent**

In [ ]:
await assistant.load_state(state_agent)

### **Managing states in Teams** _ before is single Agent

In [ ]:
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.messages import TextMessage
from autogen_core import CancellationToken
from autogen_agentchat.conditions import MaxMessageTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from dotenv import load_dotenv
import os
import asyncio
load_dotenv()

apikey = os.getenv("OPENAI_API_KEY")

modelclient = OpenAIChatCompletionClient(api_key=apikey, model="gpt-4o-mini")

In [ ]:
agent1 = AssistantAgent(name="writer_1",
                        model_client=modelclient,
                        system_message="You are a contemporary writer. Give the output in less than 30 words."
                        )

agent2 = AssistantAgent(name="writer_2",
                        model_client=modelclient,
                        system_message="You are a poet. Give the output in less than 30 words."
                        )

agent_team = [agent1, agent2]
termination_condition = MaxMessageTermination(3)

team_orchestration = RoundRobinGroupChat(agent_team,
                                         termination_condition=termination_condition
                                         )

In [ ]:
await Console(team_orchestration.run_stream(task="Tell me a stroy about a wise fox."))

**Save State**

In [ ]:
team_state = await team_orchestration.save_state()
print("Team state saved: ", team_state)

In [ ]:
await team_orchestration.reset()

In [ ]:
await team_orchestration.load_state(team_state)

## **Note single agent output is loaded to single agent only Teams Agent is loaded only to teams**

In [ ]:
stream = team_orchestration.run_stream(task="what was the last line of the poen wrote.")
await Console(stream)

### **Saving and Loading Team State form file** 

In [ ]:
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.messages import TextMessage
from autogen_core import CancellationToken
from autogen_agentchat.conditions import MaxMessageTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from dotenv import load_dotenv
import os
import asyncio
load_dotenv()

apikey = os.getenv("OPENAI_API_KEY")

modelclient = OpenAIChatCompletionClient(api_key=apikey, model="gpt-4o-mini")

In [ ]:
agent1 = AssistantAgent(name="writer_1",
                        model_client=modelclient,
                        system_message="You are a contemporary writer. Give the output in less than 30 words."
                        )

agent2 = AssistantAgent(name="writer_2",
                        model_client=modelclient,
                        system_message="You are a poet. Give the output in less than 30 words."
                        )

agent_team = [agent1, agent2]
termination_condition = MaxMessageTermination(3)

team_orchestration = RoundRobinGroupChat(agent_team,
                                         termination_condition=termination_condition
                                         )

In [ ]:
await Console(team_orchestration.run_stream(task="Tell me a stroy about a wise fox."))

In [ ]:
team_state = await team_orchestration.save_state()
print("Team state saved: ", team_state)

In [ ]:
team_state

### **write state to a file**

In [ ]:
import json

with open("team_state.json", "w") as f:
    json.dump(team_state, f)

### **Load state to a file**

In [ ]:
with open("team_state.json", "r") as f:
    loaded_team_state = json.load(f)

In [ ]:
await team_orchestration.load_state(loaded_team_state)
print("Team state loaded from file.", loaded_team_state)